# Deep learning: end-to-end by using TensorFlow Keras, Hyperopt and MLflow

###[Banafsheh Hassani](https://www.linkedin.com/in/banafsheh-hassani-7b063a129/)

###[More Projects](https://github.com/BanafshehHassani)

[Reference](https://scikit-learn.org/0.19/auto_examples/text/document_classification_20newsgroups.html)
[Reference](https://docs.databricks.com/_static/notebooks/getting-started/get-started-keras-dbr7ml.html)

# Deep learning: end-to-end by using TensorFlow Keras, Hyperopt and MLflow

 Data : Fetch_california_housing from scikit-learn

 Use TensorFlow Keras, Hyperopt, and MLflow to develop a deep learning model on Fetch_california_housing dataset.

# Steps: 
* Data loading and preprocess
* Build a neural network model by TensorFlow Keras plus view training by inline TensorBoard
* Perform automated hyperparameter tuning with Hyperopt and MLflow plus use autologging to save results
* Use best hyperparameters set to creat a final model 
* Register model in MLflow plus use the model for make predictions

In [0]:
pip install --upgrade tensorflow

In [0]:
pip install mlflow 

In [0]:
pip install hyperopt

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
import mlflow.tensorflow

## Load and data preprocessing
California Housing dataset from `scikit-learn`.

In [0]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()


# Split 80/20 train-test

In [0]:
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                    cal_housing.target,
                                                    test_size=0.2)

### Scale features
Feature scaling by `scikit-learn` function `StandardScaler`.

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build model 
 View TensorBoard in notebook

### Build a neural network

In [0]:
def create_model():
  model = Sequential()
  model.add(Dense(20, input_dim=8, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

# Compile the model

In [0]:
model = create_model()

model.compile(loss="mse",
              optimizer="Adam",
              metrics=["mse"])

# Create callbacks

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

experiment_log_dir = "/dbfs/Banafshhassani@gmail.com/tb"
checkpoint_path = "/dbfs/Banafshhassani@gmail.com/keras_checkpoint_weights.ckpt"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_log_dir)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="loss", mode="min", patience=3)

history = model.fit(X_train, y_train, validation_split=.2, epochs=35, callbacks=[tensorboard_callback, model_checkpoint, early_stopping])

# TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir $experiment_log_dir

In [0]:
dbutils.tensorboard.start(experiment_log_dir)

### Evaluate model on test dataset

In [0]:
model.evaluate(X_test, y_test)

# Hyperparameter tuning by Hyperopt and MLflow

### Create a neural network model by using variables for number of nodes on hidden layers

In [0]:
def create_model(n):
  model = Sequential()
  model.add(Dense(int(n["dense_l1"]), input_dim=8, activation="relu"))
  model.add(Dense(int(n["dense_l2"]), activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

### Create Hyperopt objective function

In [0]:
from hyperopt import fmin, hp, tpe, STATUS_OK, SparkTrials

def runNN(n):
  # Import tensorflow 
  import tensorflow as tf
  
  # Log run information with mlflow.tensorflow.autolog()
  mlflow.tensorflow.autolog()
  
  model = create_model(n)

  # Select optimizer
  optimizer_call = getattr(tf.keras.optimizers, n["optimizer"])
  optimizer = optimizer_call(learning_rate=n["learning_rate"])
 
  # Compile model
  model.compile(loss="mse",
                optimizer=optimizer,
                metrics=["mse"])

  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2)

  # Evaluate the model
  score = model.evaluate(X_test, y_test, verbose=0)
  obj_metric = score[0]  
  return {"loss": obj_metric, "status": STATUS_OK}

### Define Hyperopt search space

In [0]:
space = {
  "dense_l1": hp.quniform("dense_l1", 10, 30, 1),
  "dense_l2": hp.quniform("dense_l2", 10, 30, 1),
  "learning_rate": hp.loguniform("learning_rate", -5, 0),
  "optimizer": hp.choice("optimizer", ["Adadelta", "Adam"])
 }

### Create the `SparkTrials` object

Default is the number of available Spark executors without specify a parallelism argument.

In [0]:
spark_trials = SparkTrials()

### Perform hyperparameter tuning

In [0]:
with mlflow.start_run():
  best_hyperparam = fmin(fn=runNN, 
                         space=space, 
                         algo=tpe.suggest, 
                         max_evals=30, 
                         trials=spark_trials)

## Use the best set of hyperparameters to build a final model

In [0]:
import hyperopt

print(hyperopt.space_eval(space, best_hyperparam))

In [0]:
first_layer = hyperopt.space_eval(space, best_hyperparam)["dense_l1"]
second_layer = hyperopt.space_eval(space, best_hyperparam)["dense_l2"]
learning_rate = hyperopt.space_eval(space, best_hyperparam)["learning_rate"]
optimizer = hyperopt.space_eval(space, best_hyperparam)["optimizer"]

### Get optimizer plus update with learning_rate value

In [0]:
optimizer_call = getattr(tf.keras.optimizers, optimizer)
optimizer = optimizer_call(learning_rate=learning_rate)

In [0]:
def create_new_model():
  model = Sequential()
  model.add(Dense(first_layer, input_dim=8, activation="relu"))
  model.add(Dense(second_layer, activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

In [0]:
new_model = create_new_model()
  
new_model.compile(loss="mse",
                optimizer=optimizer,
                metrics=["mse"])

When `autolog()` is active, it needs to end the run that was started before starting plus autologging a new run.

In [0]:
mlflow.end_run()

In [0]:
import matplotlib.pyplot as plt

mlflow.tensorflow.autolog()

with mlflow.start_run() as run:
  
  history = new_model.fit(X_train, y_train, epochs=35, callbacks=[early_stopping])
  
  # Save the run information to register the model later
  kerasURI = run.info.artifact_uri
  
  # Evaluate model on test dataset and log result
  mlflow.log_param("eval_result", new_model.evaluate(X_test, y_test)[0])
  
  # Plot predicted vs known values for a quick visual check of the model and log the plot as an artifact
  keras_pred = new_model.predict(X_test)
  plt.plot(y_test, keras_pred, "o", markersize=2)
  plt.xlabel("observed value")
  plt.ylabel("predicted value")
  plt.savefig("kplot.png")
  mlflow.log_artifact("kplot.png") 

## Register the model in MLflow and use the model to make predictions

* Note: if you register model with same name before you will experince error.

In [0]:
'''
import time

model_name = "cal_housing_keras"
model_uri = kerasURI+"/model"
new_model_version = mlflow.register_model(model_uri, model_name)

# Registering the model takes a few seconds, so add a delay before continuing with the next cell
time.sleep(5)
'''

### Load the model for inference and make predictions

In [0]:
'''
keras_model = mlflow.keras.load_model(f"models:/{model_name}/{new_model_version.version}")

keras_pred = keras_model.predict(X_test)
keras_pred
'''

## Clean up
To stop TensorBoard

In [0]:
dbutils.tensorboard.stop()